# Linear Mixed Model

In [1]:
from sklearn.preprocessing import StandardScaler

from scripts.utils import *
import statsmodels.formula.api as smf

In [2]:
# set path
data_path = get_path('dataframes')

Load data

In [3]:
threshold_data = pd.read_pickle(os.path.join(data_path, 'thresholds-3AFC_freqs.pkl'))[
    ['participant', 'paradigm', 'pred', 'mean_threshold']].drop_duplicates()

p50_data = pd.read_pickle(os.path.join(data_path, 'sigmoid_data.pkl'))[
    ['participant', 'paradigm', 'pred', 'distance_p50']].drop_duplicates()

# FAR_data = pd.read_pickle(os.path.join(data_path, 'false_alarm_rates.pkl'))[
#     ['participant', 'paradigm', 'pred', 'false_alarm_rate']].drop_duplicates()

gmsi_data = load_gmsi_results()

age = load_age_info()

In [4]:
# Merge everything
all_data = pd.merge(threshold_data, p50_data, on=['paradigm', 'pred', 'participant'])
# all_data = pd.merge(all_data, FAR_data, on=['paradigm', 'pred', 'participant'])
all_data = pd.merge(all_data, gmsi_data, on='participant')
all_data = pd.merge(all_data, age, on='participant')

all_data.head()

,participant,paradigm,pred,mean_threshold,distance_p50,gmsi,age
0,cbtxie,3AFC,3AFC,-2.154545,-5.970877,62,31
1,cbtxie,Bayesian,Bayesian,2.552655,0.764187,62,31
2,cbtxie,Cluster,both,2.479225,-0.184336,62,31
3,cbtxie,Cluster,frequency,1.429287,-1.157223,62,31
4,cbtxie,Cluster,none,1.792861,-0.296288,62,31


In [5]:
scaler = StandardScaler()

# Select only numerical columns
numerical_cols = ['mean_threshold', 'distance_p50', 'age', 'gmsi']  # add or remove according to your dataset

# Fit and transform the data
all_data_standardized = all_data.copy()
all_data_standardized[numerical_cols] = scaler.fit_transform(all_data[numerical_cols])


#### Fit Linear Mixed Model

In [6]:
# sort categories to set reference level
map_task = {'Bayesian': '0_Randomized', 'Continuous': '1_Continuous', 'Cluster': '2_Cluster', '3AFC': '3_3AFC'}
all_data['paradigm'] = all_data['paradigm'].map(map_task)
map_pred = {'Bayesian': '0_randomized', 'none': '1_R', 'time': '2_T', 'frequency': '3_F', 'both': '4_FT', '3AFC': '5_3AFC'}
all_data['pred'] = all_data['pred'].map(map_pred)

# Fit a mixed effects model
model = smf.mixedlm("mean_threshold ~ paradigm*pred*age*gmsi", all_data, groups=all_data['participant'])

# Fit model and print results
result = model.fit()
print(result.summary())

# Predicted values
all_data['predicted'] = result.fittedvalues

# Residuals
all_data['residuals'] = result.resid

LinAlgError: Singular matrix